## Section 2.4
So we've seen the benefits of $\epsilon$-greedy strategies, but let's take a second to deal with the implementation. 

Our naive approach to estimating the rewards of each action requires $O(n)$ memory and computation, because, assuming $t=n$, we need to average the last $n$ rewards (recall that $Q_t$, shorthard for $Q_t(a)$ is the estimate of some action's true reward at time $t$):
$$
Q_n = \frac{R_1 + ... + R_{n-1}}{n-1}
$$
That could end up being a lot of additions and rewards stored! In fact, we could even encounter numerical instability issues if the sum of rewards got very large. So, we find a way to calculate $Q_n$ incrementally:
$$
\begin{align}
Q_n &= \frac{R_1 + ... + R_{n-1}}{n-1} \\
&= \frac{R_1 + ... + R_{n-2}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-2} \frac{R_1 + ... + R_{n-2}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-1} \frac{R_1 + ... + R_{n-2}}{n-2} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-1} Q_{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-1} Q_{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{(n-1)Q_{n-1} - Q_{n-1}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= Q_{n-1} - \frac{Q_{n-1}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= Q_{n-1} + \frac{1}{n-1}[R_{n-1} - Q_{n-1}] \\
\end{align}
$$
Wowza! So this gives us a recursive formula for computing the reward in constant time and memory. 

It also looks a bit like gradient descent, with a step size of $1/t$. In fact, it is! It's clearer if we look at the case for $t=n+1$ and rearrange a bit:
$$
Q_{n+1} = Q_n + \frac{1}{n}[R_n - Q_n]
$$
Sutton & Barto make this clear in the book:
$$
\text{NewEstimate} = \text{OldEstimate} + \text{StepSize}[\text{NewEstimate} - \text{OldEstimate}].
$$
Of course, this begs the question: what happens if we change the step size?

## Section 2.5
TBD